<a href="https://colab.research.google.com/github/olumor10/Algoritmo_PSO/blob/main/Algoritmo_PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='cyan'>**Otimização por Enxame de Partículas - PSO**</font>
```
BIBLIOGRAFIA:
Haupt, R. L. and Haupt, S. E. Practical Genetic Algorithms. Wiley-Interscience, 2a Edição. 2004.
```


<font color="lime">**Equipe:**</font>

Allan Jorge

Rafael Azevedo

Rômulo Rodrigues

---
## <font color='cyan'>Função de custo e limites do espaço de busca</font>
$$ f(x, y) = -e^{-0.2 \sqrt{x^2 + y^2} + 3 (\cos(2x) + \sin(2y))}, \text{ sujeito a } -5 ≤ x ≤ 5 \text{ e } -5 ≤ y ≤ 5.$$

In [ ]:
#Bibliotecas
import numpy as np
import plotly.graph_objects as go
import pandas as pd

# Definição da função de custo
def funcao_custo(posicoes):
    # return posicoes[:, 1] * np.sin(4 * posicoes[:, 0]) + 1.1 * posicoes[:, 0] * np.sin(2 * posicoes[:, 1])  #F8   (x,y) = [0, 10]
    # return -np.exp(0.2 * np.sqrt((posicoes[:, 0] - 1) ** 2 + (posicoes[:, 1] - 1) ** 2) + (np.cos(2 * posicoes[:, 0]) + np.sin(2 * posicoes[:, 0]))) #F15 modificada
    return -np.exp(-0.2 * np.sqrt((posicoes[:, 0])**2 + (posicoes[:, 1]**2)) + 3 * (np.cos(2 * posicoes[:, 0]) + np.sin(2 * posicoes[:, 1])))  #F15 real (x,y) = [-5, 5]

    # A função é definida para receber um array bidimensional, onde cada linha representa a posição de uma partícula no espaço de busca e as colunas representam as coordenadas (x, y).
      # posicoes[:, 0] corresponde à coordenada x.
      # posicoes[:, 1] corresponde à coordenada y.

# limites das variáveis (espaço de busca).
lim_inf = -5    # limite inferior
lim_sup = 5     # limite superior

---
## <font color='cyan'>Parâmetros iniciais</font>

In [ ]:
tamanho_enxame = 10  # Define o número de partículas no enxame.

dimensao = 2  # Define a dimensão do espaço de busca (x e y).

max_iteracoes = 100  # Número máximo de iterações.

cognitivo = 1  # O parâmetro cognitivo controla o quanto uma partícula é influenciada por sua melhor posição individual encontrada até o momento.

social = 4 - cognitivo  # O parâmetro social controla o quanto uma partícula é influenciada pela melhor posição global do enxame.

---
## <font color='cyan'>Inicializando o enxame e velocidades</font>

In [ ]:
posicoes = np.random.rand(tamanho_enxame, dimensao) * (lim_sup - lim_inf) + lim_inf     # inicializa as posições iniciais das partículas no espaço de busca dentro dos limites especificados.
velocidades = np.random.rand(tamanho_enxame, dimensao)  # Gera uma matriz de números aleatórios no intervalo [0, 1] e cada elemento representa a velocidade inicial de uma partícula em uma dimensão específica.
custos = funcao_custo(posicoes)  # Avalia o custo para cada linha (partícula) e retorna um array unidimensional com os custos correspondentes.

# Para plotar o diagrama de Pareto
todos_x_local = [posicoes[:, 0]]
todos_y_local = [posicoes[:, 1]]
todos_custo_local = [custos]

In [ ]:
# Exibir a posição inicial das partículas do enxame
x = np.linspace(lim_inf, lim_sup, 200)
y = np.linspace(lim_inf, lim_sup, 200)
X, Y = np.meshgrid(x, y)

XY = np.c_[X.ravel(), Y.ravel()]
Z = funcao_custo(XY).reshape(X.shape)

fig = go.Figure()
fig.add_trace(go.Contour(
    z=Z,
    x=x,
    y=y,
    contours_coloring='lines',
    colorscale='Viridis',
    line_width=1,
    colorbar=dict(title='Custo', thickness=15)
))
fig.add_trace(go.Scatter(
    x=posicoes[:, 0],
    y=posicoes[:, 1],
    mode='markers',
    marker=dict(size=10, color='red', symbol='circle'),
    name='Partícula'
))
fig.update_layout(
    title="Posição inicial do Enxame",
    xaxis_title="x",
    yaxis_title="y",
    margin=dict(l=10, r=10, b=10, t=50),
    width=600,
    height=400,
    xaxis_range=[lim_inf, lim_sup],
    yaxis_range=[lim_inf, lim_sup]
)
fig.show()

---
## <font color='cyan'>Inicializando o melhor custo local e global</font>

In [ ]:
melhor_custo_iteracao = [custos.min()]  # Melhor custo na iteração atual
custo_medio_iteracao = [custos.mean()]  # Custo médio do enxame em cada iteração
melhor_custo_global = [melhor_custo_iteracao[0]]  # Melhor custo global até agora
melhor_posicao_global = posicoes[custos.argmin()]  # Melhor posição global até agora
melhores_posicoes_locais = posicoes.copy()  # Melhor posição local de cada partícula
melhores_custos_locais = custos.copy()  # Melhor custo local de cada partícula

---
## <font color='cyan'>Início das iterações</font>

In [ ]:
for iteracao in range(max_iteracoes):
    # Atualiza a inércia
    inercia = (max_iteracoes - iteracao) / max_iteracoes
    # A inércia é um fator que controla a influência da velocidade anterior no cálculo da nova velocidade de cada partícula.
    # A inércia é reduzida ao longo das iterações, indo de um valor inicial (1) até um valor final próximo de 0.
    # No início do processo, uma alta inércia favorece a exploração (movimentos maiores).
    # No final do processo, uma baixa inércia favorece a exploração local (movimentos menores), ajudando o enxame a se concentrar em refinar soluções.

    # Números aleatórios para atualização das velocidades
    aleatorio1 = np.random.rand(tamanho_enxame, dimensao)   # Relaciona-se ao componente cognitivo (a partícula tenta retornar à sua melhor posição local).
    aleatorio2 = np.random.rand(tamanho_enxame, dimensao)   # Relaciona-se ao componente social (a partícula é atraída para a melhor posição global).

    # Atualiza as velocidades
    velocidades = (
        inercia * velocidades   # Controla o quanto do movimento anterior a partícula deve reter.
        + cognitivo * aleatorio1 * (melhores_posicoes_locais - posicoes)   # Incentiva cada partícula a explorar novamente regiões onde ela já encontrou bons resultados.
        + social * aleatorio2 * (melhor_posicao_global - posicoes)  # Incentiva a colaboração entre partículas, guiando-as para a melhor solução coletiva.
    )

    # Atualiza as posições das partículas
    posicoes = posicoes + velocidades
    posicoes = np.clip(posicoes, lim_inf, lim_sup)  # Limita as partículas ao intervalo especificado.

    # Avalia os custos das novas posições
    custos = funcao_custo(posicoes)

    # Atualiza o melhor custo e posição local
    melhorou = custos < melhores_custos_locais  # Realiza uma comparação elemento a elemento entre os custos atuais e os melhores custos locais. O resultado é um array booleano de mesmo tamanho que custos
    melhores_custos_locais[melhorou] = custos[melhorou]   # Atualiza os melhores custos locais para as partículas que encontraram melhores posições.
    melhores_posicoes_locais[melhorou] = posicoes[melhorou]   # Atualiza as melhores posições locais para as partículas que encontraram melhores custos.

    # Atualiza o melhor custo e posição global
    if custos.min() < melhor_custo_global[-1]:
        melhor_custo_global.append(custos.min())   # Adiciona o valor do menor custo global já encontrado.
        melhor_posicao_global = posicoes[custos.argmin()]  # Atualiza melhor_posicao_global com a posição da partícula que encontrou o menor custo.
    else:
        melhor_custo_global.append(melhor_custo_global[-1]) # Adiciona novamente o menor custo global se o custo atual for maior que o anterior. (Para plotar a curva do melhor custo global)

    # Salva os dados para plotagem
    melhor_custo_iteracao.append(custos.min())
    custo_medio_iteracao.append(custos.mean())

    # Para plotar o diagrama de Pareto
    todos_x_local.append(posicoes[:, 0])
    todos_y_local.append(posicoes[:, 1])
    todos_custo_local.append(custos)

    # Exibe o progresso
    # print(f"Iteração {iteracao + 1}: Melhor custo global = {melhor_custo_global}")

# Para plotar o diagrama de Pareto
todos_x = np.concatenate(todos_x_local)
todos_y = np.concatenate(todos_y_local)
todos_custo = np.concatenate(todos_custo_local)

In [ ]:
# Posição do Enxame por iteração
num_interacao = 100     # Define o numero da iteração para analisar a posição do exame em uma iteração específica.

if num_interacao <= max_iteracoes:
    tabela = pd.DataFrame({'x': todos_x_local[num_interacao], 'y': todos_y_local[num_interacao], 'custo': todos_custo_local[num_interacao]})
    tabela_ordenada = tabela.sort_values(by=['custo'], ascending=True)
    print(f"Enxame da iteração {num_interacao} ordenado pelo menor custo.")
    print(tabela_ordenada)
else:
    print(f"(num_interacao) deve ser menor que {max_iteracoes}!")

fig = go.Figure()
fig.add_trace(go.Contour(
    z=Z,
    x=x,
    y=y,
    contours_coloring='lines',
    colorscale='Viridis',
    line_width=1,
    colorbar=dict(title='Custo', thickness=15)
))
fig.add_trace(go.Scatter(
    x=todos_x_local[num_interacao],
    y=todos_y_local[num_interacao],
    mode='markers',
    marker=dict(size=10, color='red', symbol='circle'),
    name='Partícula'
))
fig.update_layout(
    title=f"Posição do Enxame na Iteração {num_interacao}",
    xaxis_title="x",
    yaxis_title="y",
    margin=dict(l=10, r=10, b=10, t=50),
    width=600,
    height=400,
    xaxis_range=[lim_inf, lim_sup],
    yaxis_range=[lim_inf, lim_sup]
)
fig.show()

Enxame da iteração 100 ordenado pelo menor custo.
              x         y       custo
3 -4.764350e-07 -2.339514 -252.251832
1  7.016730e-07 -2.339512 -252.251832
9 -5.144799e-05 -2.339512 -252.251828
4 -4.331958e-05 -2.339485 -252.251827
6 -7.232621e-05 -2.339512 -252.251824
5 -5.510163e-04 -2.339529 -252.251369
0 -3.178668e-06 -2.338923 -252.251285
7 -1.776126e-05 -2.341279 -252.247179
8  1.300271e-01 -2.336941 -227.874062
2 -1.961478e+00 -1.597609   -0.084162


---
## <font color='cyan'>Resultados</font>

### Numéricos

In [ ]:
print(f"Melhor custo global: {min(melhor_custo_global)}")
print(f"Melhor posição global: x = {melhor_posicao_global[0]}, y = {melhor_posicao_global[1]}")

Melhor custo global: -252.25183210700655
Melhor posição global: x = -4.764350438200507e-07, y = -2.339513691239589


### Gráficos 📊

Evolução dos custos

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(max_iteracoes + 1)),
    y=melhor_custo_iteracao,
    mode='lines',
    name='Melhor custo'
))
fig.add_trace(go.Scatter(
    x=list(range(max_iteracoes + 1)),
    y=custo_medio_iteracao,
    mode='lines',
    name='Custo médio',
))
fig.add_trace(go.Scatter(
    x=list(range(max_iteracoes + 1)),
    y=melhor_custo_global,
    mode='lines',
    name='Melhor custo global',
    line=dict(dash='dash')
))
fig.update_layout(
    title="Evolução dos custos",
    xaxis_title="Iteração",
    yaxis_title="Custo",
    margin=dict(l=10, r=10, b=10, t=50),
    width=600,
    height=400,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99)
)
fig.show()

Diagrama de Pareto

In [ ]:
fig = go.Figure(data=[go.Scatter(
    x=todos_x,
    y=todos_y,
    mode='markers',
    marker=dict(
        color=todos_custo,
        colorscale='Viridis',
        colorbar=dict(
            title="Custo",
            thickness=15
        ),
        symbol='star',
        size=10),
)])
fig.update_layout(
    xaxis_title='Soluções de X',
    yaxis_title='Soluções de Y',
    title='Diagrama de Pareto',
    margin=dict(l=10, r=10, b=10, t=50),
    xaxis_range=[lim_inf, lim_sup],
    yaxis_range=[lim_inf, lim_sup],
    width=600,
    height=400,
)
fig.show()

Diagrama de Superfície



In [ ]:
fig = go.Figure(data=[go.Surface(
    z=Z,
    x=x,
    y=x,
    colorscale='Viridis',
    colorbar=dict(title='Custo', thickness=15)
)])
fig.add_trace(go.Scatter3d(
    x=[melhor_posicao_global[0]],
    y=[melhor_posicao_global[1]],
    z=[melhor_custo_global[-1]],
    mode='markers+text',
    marker=dict(size=10, color='red', symbol='circle'),
    text=[f"Custo = {melhor_custo_global[-1]:.6f}"],
    textposition="bottom center",
    name='Melhor custo',
    textfont=dict(color="red")
))
fig.update_layout(
    title="Gráfico de superfície da função de custo",
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="f(x, y)",
        aspectratio=dict(x=1, y=1, z=1)  # Ajusta a proporção dos eixos manualmente
    ),
    margin=dict(l=10, r=10, b=10, t=50),
    width=600,
    height=400,
)
fig.show()

## <font color='cyan'>Animação do Enxame</font>

In [ ]:
vel_anima = 500 #normal 400 ms

fig = go.Figure()
contour = go.Contour(
    z=Z,
    x=x,
    y=y,
    contours_coloring='lines',
    colorscale='Viridis',
    line_width=1,
    colorbar=dict(title='Custo', thickness=15),
    name='Contour'

)

scatter = go.Scatter(
    x=todos_x_local[0],  # Começa com as posições iniciais
    y=todos_y_local[0],
    mode='markers',
    marker=dict(size=12, color='red', symbol='circle'),
    name='Partícula'
)

fig.add_trace(contour)
fig.add_trace(scatter)

# Cria os frames da animação
frames = []
for i in range(max_iteracoes + 1):
    frame = go.Frame(
        data=[go.Scatter(
                  x=todos_x_local[i], # atualiza as posições
                  y=todos_y_local[i])
              ],
        traces=[1],
        name=f'frame{i}'  # Nome do frame para o slider
    )
    frames.append(frame)

fig.frames = frames

fig.update_layout(
    title="Animação do Enxame de Partículas",
    xaxis_title="x",
    yaxis_title="y",
    margin=dict(l=10, r=10, b=10, t=50),
    xaxis_range=[lim_inf, lim_sup],
    yaxis_range=[lim_inf, lim_sup],
    width=600,
    height=600,
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="Play", method="animate", args=[None,  {"frame": {"duration": vel_anima, "redraw": True}, "fromcurrent": True}]),
                dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}]),
            ],
        x=-0.06,
        y=-0.3,
        xanchor="left",
        yanchor="bottom",
        )
    ],
    sliders=[{
        'active':
        0,
        'yanchor':
        'top',
        'xanchor':
        'left',
        'currentvalue': {
            'font': {
                'size': 15
            },
            'prefix': 'Iteração: ',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {
            'duration': vel_anima,
            'easing': 'cubic-in-out'
        },
        'pad': {
            'b': 10,
            't': 50
        },
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': [{
            'args': [[f'frame{k}'], {
                'frame': {
                    'duration': vel_anima,
                    'redraw': True
                },
                'mode': 'immediate',
                'transition': {
                    'duration': vel_anima
                }
            }],
            'label': k,
            'method': 'animate'
        } for k in range(max_iteracoes + 1)]
    }]
)

fig.show()